In [ ]:
! pip install cairosvg

In [2]:
! gzip -d /content/kanjivg-20220427.xml.gz   # This extracts, and then deletes the things.

In [72]:
## Trying XML file tree thing.
import xml.etree.ElementTree as ET
tree = ET.parse('kanjidic2.xml')
root = tree.getroot()

kanji = dict()
kanji_list = []
mn_list = []

for ch in root.iter('character'):  # It seems that the 0 index element is just some metadata, all entries apart from that are Kanji Characters.
    n_mn_list = []
    for lt in ch.iter('literal'):
      txt = lt.text
      kanji_list.append(lt.text)

    for mn in ch.iter('meaning'):
      n_mn_list.append(mn.text)

    mn_list.append(n_mn_list)
    kanji[txt] = " ".join(n_mn_list)


assert len(kanji_list) == len(mn_list)
print(kanji)

{'亜': 'Asia rank next come after -ous Asie suivant sub- sous- pref. para indicar venir después de Asia Ásia próxima o que vem depois -ous', '唖': 'mute dumb', '娃': 'beautiful', '阿': 'Africa flatter fawn upon corner nook recess flatter flagorner Afrique coin recoin A fonética África meandro adular halagar', '哀': 'pathetic grief sorrow pathos pity sympathize pitoyable peine chagrin pitié pathétique compatir compasión lástima miseria piedad pena compadecerse de patético pesar pena emoção compaixão solidariesar', '愛': 'love affection favourite amour affection favori amor afecto favorito amor afeição favorito', '挨': 'approach draw near push open empujar acercarse', '姶': 'good-looking quiet', '逢': 'meeting tryst date rendezvous encuentro cita encuentro casual encontrarse reunirse citarse verse por casualidad', '葵': 'hollyhock malva real malva loca', '茜': 'madder red dye Turkey red tinte rojo rojo brillante', '穐': 'autumn', '悪': 'bad vice rascal false evil wrong mauvais vice vaurien faux mal m

In [65]:
import typing

T = typing.TypeVar("T")


class OrderedSet(typing.MutableSet[T]):
    """A set that preserves insertion order by internally using a dict."""

    def __init__(self, iterable: typing.Iterable[T]):
        self._d = dict.fromkeys(iterable)

    def add(self, x: T) -> None:
        self._d[x] = None

    def discard(self, x: T) -> None:
        self._d.pop(x, None)

    def __contains__(self, x: object) -> bool:
        return self._d.__contains__(x)

    def __len__(self) -> int:
        return self._d.__len__()

    def __iter__(self) -> typing.Iterator[T]:
        return self._d.__iter__()

    def __str__(self):
        return f"{{{', '.join(str(i) for i in self)}}}"

    def __repr__(self):
        return f"<OrderedSet {self}>"

In [68]:
## Trying XML file tree thing. Done.

import cairosvg
import xml.etree.ElementTree as ET

tree = ET.parse('kanjivg-20220427.xml')
root = tree.getroot()

paths = []
nested_paths = []
ids = OrderedSet([])

for graphics in root.iter("g"):  # Find all path elements
    nested_paths = []
    for path in graphics.iter('path'):

      nested_paths.append(path.get("d"))
      ids.add(path.get("id")[4:9])

    # This will be very useful when we draw from this.
    if len(nested_paths) == 1:
      paths.append(nested_paths[0])
      ids.add(graphics.get("id")[4:9])

    #ids.append(graphics.get("id")[4:])  # This is an important step when we actually host the the data !!


In [69]:
print(f"Lengths are - Meanings : {len(kanji_list)} and Ids : {len(ids)}")

print(ids) # Now we need to corrospond the ids to the actual meaning.

Lengths are - Meanings : 13108 and Ids : 6761
{00021, 0002c, 0002e, 00030, 00031, 00032, 00033, 00034, 00035, 00036, 00037, 00038, 00039, 0003a, 0003b, 0003f, 00041, 00042, 00043, 00044, 00045, 00046, 00047, 00048, 00049, 0004a, 0004b, 0004c, 0004d, 0004e, 0004f, 00050, 00051, 00052, 00053, 00054, 00055, 00056, 00057, 00058, 00059, 0005a, 00061, 00062, 00063, 00064, 00065, 00066, 00067, 00068, 00069, 0006a, 0006b, 0006c, 0006d, 0006e, 0006f, 00070, 00071, 00072, 00073, 00074, 00075, 00076, 00077, 00078, 00079, 0007a, 02e8c, 02e8d, 02e95, 02e97, 02ea1, 02ea4, 02ea6, 02ea8, 02ec3, 02ecc, 02ecf, 02ed6, 03001, 03002, 03005, 03006, 03041, 03042, 03043, 03044, 03045, 03046, 03047, 03048, 03049, 0304a, 0304b, 0304c, 0304d, 0304e, 0304f, 03050, 03051, 03052, 03053, 03054, 03055, 03056, 03057, 03058, 03059, 0305a, 0305b, 0305c, 0305d, 0305e, 0305f, 03060, 03061, 03062, 03063, 03064, 03065, 03066, 03067, 03068, 03069, 0306a, 0306b, 0306c, 0306d, 0306e, 0306f, 03070, 03071, 03072, 03073, 03074, 0

In [10]:
import cairosvg

def svg_path_to_png(svg_path, output_png_path):

    if type(svg_path) is list:
      svg_content = '<svg xmlns="http://www.w3.org/2000/svg">{}</svg>'.format(
        "".join('<path d="{}"/>'.format(path) for path in svg_path))

    elif type(svg_path) is str:
      svg_content = f'<svg xmlns="http://www.w3.org/2000/svg"><path d="{svg_path}"/></svg>'

    cairosvg.svg2png(bytestring=svg_content.encode(), write_to=output_png_path, output_width=128, output_height=128)


for i in range(len(paths)):
  svg_path = paths[i]
  output_png_path = f"/content/kanji_png/{ids[i]}.png"
  svg_path_to_png(svg_path, output_png_path)

In [11]:
import os
from google.colab import files
import zipfile

# Specify the folder path you want to zip
folder_path = '/content/kanji_png'

# Create a zip file in Colab
with zipfile.ZipFile('kanji_png.zip', 'w') as zip_ref:
  for root, _, files in os.walk(folder_path):
    for file in files:
      zip_ref.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))